In [ ]:
import warnings
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window
from delta.tables import DeltaTable

warnings.filterwarnings("ignore", category=FutureWarning)
# Create SparkSession
spark = (SparkSession.builder.appName("DeltaSession")
            .config("spark.executor.memory", "16G")
            .config("spark.driver.memory", "8G")
            #.config("spark.driver.maxResultSize", "2G")
            .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0")
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            .getOrCreate()
        )

In [ ]:
# Define the schema for a DataFrame
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True),
])

# Create a DataFrame using the schema
data = [(1, "Alice", 25, "New York"), 
        (2, "Bob", 30, "San Francisco"), 
        (3, "Charlie", 35, "Chicago")]

df = spark.createDataFrame(data, schema)

# Show the DataFrame
df.show()

df.write.format("delta").saveAsTable("optimization22")

In [ ]:
spark.sql("DESCRIBE optimization22").toPandas()

In [ ]:
spark.sql("DESCRIBE HISTORY optimization22").toPandas()

In [ ]:
spark.sql("SHOW TABLES").toPandas()

In [ ]:
%%time

df.count()

In [ ]:
df_dt = DeltaTable.forPath(spark, 'spark-warehouse/optimization22')

df_dt.toDF().toPandas()

## Compaction

In [ ]:
opt = df_dt.optimize().executeCompaction()

In [ ]:
opt.toPandas()

In [ ]:
opt.select("metrics.*").toPandas()

## Vacuum to see the compaction more clearly

In [ ]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

In [ ]:
df_dt.vacuum(0)

In [ ]:
# If you have a large amount of data and only want to optimize a subset of it, you can specify an optional partition predicate using `where`
deltaTable.optimize().where("date='2021-11-18'").executeCompaction()

In [ ]:
# mini ttest

In [ ]:
df = spark.range(0, 5)

## Z-Order

In [ ]:
dt = DeltaTable.forPath(spark, 'extract/09delta')

dt.toDF().toPandas()

In [ ]:
opt = spark.sql("OPTIMIZE optimization ZORDER BY (id)")

In [ ]:
# python
dt.optimize().executeZOrderBy('id')

In [ ]:
opt.toPandas()

In [ ]:
opt.collect()

In [ ]:
opt.select("metrics.*").toPandas()

In [ ]:
# If you have a large amount of data and only want to optimize a subset of it, you can specify an optional partition predicate using `where`
df_dt.optimize().where("date='2021-11-18'").executeZOrderBy(eventType)

In [ ]:
spark.sql("SELECT * FROM delta.`extract/delta`").show()

## Deletion Vector Support

In [ ]:
spark.sql("""
ALTER TABLE <table> 
SET TBLPROPERTIES (delta.enableDeletionVectors = true)
""")